# Social DistilBERT Model Using Merged Data Experiment 1.1

In [1]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from ktrain import text
import random
import warnings
from sklearn.utils import shuffle

# Set random seed
random.seed(18)
seed = 18

# Ignore warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_colwidth', None)

## 1. Loading the data and quick exploratory data analysis

In [2]:
merged_social_df = pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_Social_sentence_level_batch_1_jaccard.csv", encoding='utf-8')

# Shuffle the merged dataset
merged_social_df = shuffle(merged_social_df, random_state=seed)

# Train-test split 
training_df, test_df = train_test_split(merged_social_df, test_size=0.1, random_state=18, stratify=merged_social_df['label'])

training_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

## 2. Experimental Design

In [3]:
MAXLEN = 150

In [4]:
X = training_df['sentence']
y = training_df['label']

In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [6]:
model_name = 'distilbert-base-uncased'

In [7]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 39
	99percentile : 53


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 39
	99percentile : 62


In [8]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [9]:
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
195/195 [==============================] - 48s 199ms/step - loss: 0.3754 - accuracy: 0.8862 - val_loss: 0.3147 - val_accuracy: 0.8874
Epoch 2/5
195/195 [==============================] - 34s 172ms/step - loss: 0.2530 - accuracy: 0.8931 - val_loss: 0.2678 - val_accuracy: 0.9010
Epoch 3/5
195/195 [==============================] - 36s 180ms/step - loss: 0.1917 - accuracy: 0.9179 - val_loss: 0.2936 - val_accuracy: 0.8771
Epoch 4/5
195/195 [==============================] - 35s 178ms/step - loss: 0.1665 - accuracy: 0.9247 - val_loss: 0.3081 - val_accuracy: 0.8840
Epoch 5/5
195/195 [==============================] - 35s 175ms/step - loss: 0.1351 - accuracy: 0.9495 - val_loss: 0.3091 - val_accuracy: 0.8840


In [10]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

10/10 [==============================] - 6s 366ms/step
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       260
           1       0.48      0.48      0.48        33

    accuracy                           0.88       293
   macro avg       0.71      0.71      0.71       293
weighted avg       0.88      0.88      0.88       293



array([[243,  17],
       [ 17,  16]])

In [11]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [13]:
distillbert_test_data = test_df['sentence'].tolist()
distillbert_test_label = test_df['label'].tolist()

In [14]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [15]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [16]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 133, False Positive: 11, False Negative: 12, True Positive: 7


In [17]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.92      0.92       144
           1       0.39      0.37      0.38        19

    accuracy                           0.86       163
   macro avg       0.65      0.65      0.65       163
weighted avg       0.86      0.86      0.86       163
 



In [18]:
# distillbert_predictor.save('../../model/first_generation_distilbert_base_uncased_model_10102020') # 256 MB

In [19]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.6460160818713451
